In [28]:
import numpy as np
from pathlib import Path
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import obspy
import bisect
from pyproj import Geod
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import sys
sys.path.append('/home/Public/SiameseEQTransformer/SiameseEQT/')
from src.S_EqT_model import S_EqT_Model_create
from src.S_EqT_model import S_EqT_Model_seprate
from src.S_EqT_model import S_EqT_RSRN_Model
from src.S_EqT_model import S_EqT_HED_Model
from src.S_EqT_concate_fix_corr import S_EqT_Concate_RSRN_Model
from src.misc import get_train_list
import keras.backend
import yaml
from random import shuffle

# Load model

In [2]:
keras.backend.set_floatx('float32')
# load configuration files
cfgs = yaml.load(open('S_Test_example.yaml','r'),Loader=yaml.BaseLoader)
if_encoder_concate = int(cfgs['Model']['Encoder_concate'])
if if_encoder_concate == 1:
    encode_model, siamese_model, EqT_model = S_EqT_Concate_RSRN_Model(cfgs)
    encoder_encoded_list = cfgs['Model']['Encoder_concate_list']
    encoder_encoded_lengths = cfgs['Model']['Encoder_concate_lengths']
    encoder_encoded_channels = cfgs['Model']['Encoder_concate_channels']
        
elif int(cfgs['Model']['MODEL_RSRN'] == 0):
    encode_model, siamese_model, EqT_model = S_EqT_HED_Model(cfgs)
else:
    encode_model, siamese_model, EqT_model = S_EqT_RSRN_Model(cfgs)

if int(cfgs['Model']['LoadPretrainedModel']) == 1:
    siamese_model.load_weights(cfgs['Model']['PretrainModelPath'])

Start loading EqT model...


Start building EqT encoder model...
Start building Siamese EqT model...
Start building correlation layers...
Start sideoutput & residual layers...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1

In [3]:
siamese_model.load_weights('/home/Public/SiameseEQTransformer/SiameseEQT/models/S_Add_Noise_1023_UNI_NEW_model330000.hdf5')

In [29]:
# For P or S
# difference is velocity and trained model and encoded model
# set up model
base_time = obspy.UTCDateTime(2010,1,1)
# search params
# search distance in km
max_search_distance = 30
# template P threshold 0.5
template_P_min_threshold = 0.5
template_S_min_threshold = 0.5

VpRefMin = 3.5
VsRefMin = 2.5

P_boundary_time = 1.0
S_boundary_time = 1.0

sta_list_file_path = '/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/LAN_REAL_Use.dat'
prev_file_str = '/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/EQTRes/20100101/'
search_hdf5_prefix =  '/mnt/BYEB/LA_JAN/LA_DATA/LA_2010_Jan_processed_hdfs/'
detections_prefix = '/mnt/BYEB/LA_JAN/LA_DATA/LA_2010_Jan_detections/'

In [30]:
GEOD = Geod(ellps='WGS84')
# Rough meters/degree calculation
M_PER_DEG = (GEOD.inv(0, 0, 0, 1)[2] + GEOD.inv(0, 0, 1, 0)[2]) / 2

In [31]:
def get_search_station_list(station, station_list, max_distance):
    search_list = list()
    for t_sta in station_list:
        # skip self
        if t_sta[0] == station[0]:
            continue
        # calculate distance
        t_dis = GEOD.inv(t_sta[3],t_sta[2],station[3],station[2])[2]/1000.0
        if t_dis > max_distance:
            continue
        search_list.append(t_sta)
    return search_list

In [32]:
station_list = list()
s_eqt_dict = dict()
station_list_file = open(sta_list_file_path,'r')
sta_id = 0
for line in station_list_file.readlines():
    if len(line) < 3:
        continue
    splits = line.split(' ')
    sta_name = splits[2]+'.'+splits[3]
    
    s_eqt_dict[sta_name] = dict()
    s_eqt_dict[sta_name]['P'] = list()
    s_eqt_dict[sta_name]['S'] = list()
    s_eqt_dict[sta_name]['P_Prob'] = list()
    s_eqt_dict[sta_name]['S_Prob'] = list()
    
    sta_lat = float(splits[1])
    sta_lon = float(splits[0])
    
    station_list.append( (sta_id, sta_name, sta_lat, sta_lon) )
    sta_id += 1

In [33]:
keep_t = None
for sta_key in s_eqt_dict.keys():
    S_EqT_P_list = list()
    #print(sta_key)

    s_times = list()
    s_probs = list()
    
    prev_file = prev_file_str + '{}.S.txt'.format(sta_key)
    if os.path.exists(prev_file):
        f = open(prev_file,'r')
        for line in f.readlines():
            if len(line) > 3:
                s_times.append(float(line.split(' ')[0]))
                s_probs.append(float(line.split(' ')[1]))
        f.close()
    else:
        print(prev_file)
    #print(indexs)
    #s_times = s_times[indexs]
    #s_probs = s_probs[indexs]
    s_eqt_dict[sta_key]['S'] = s_times
    s_eqt_dict[sta_key]['S_Prob'] = s_probs

In [34]:
train_E_num = int(cfgs['Train']['Train_E_num'])
train_set_path = cfgs['Train']['Train_data_path']
step_num = 0

print_interval = int(cfgs['Train']['Train_print_interval'])
save_interval = int(cfgs['Train']['Model_save_interval'])
save_path = cfgs['Train']['Model_save_path']

RSRN_lengths = cfgs['Model']['RSRN_Encoded_lengths']
RSRN_channels = cfgs['Model']['RSRN_Encoded_channels']

In [44]:
len(s_eqt_dict['CI.RIN']['S'])

10

In [45]:
plot_dx = 0
sta_search_dx = 0
plot_num = 0
for sta in station_list:
    sta_search_dx += 1
    print('On {} of {}'.format(sta_search_dx, len(station_list)))
    # select stations to search
    h5py_base_dir = search_hdf5_prefix
    search_list = get_search_station_list(sta, station_list, max_search_distance)
    seed_csv = detections_prefix + '{}_outputs/X_prediction_results.csv'.format(sta[1][3:])
    seed_csv_file = pd.read_csv(seed_csv)
    for e_time in seed_csv_file['file_name']:
        mask = (seed_csv_file['file_name'] == e_time)
        try:
            spt_t = obspy.UTCDateTime(seed_csv_file[mask]['p_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            sst_t = obspy.UTCDateTime(seed_csv_file[mask]['s_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            spt_prob = seed_csv_file[mask]['p_probability'].values[0]
            sst_prob = seed_csv_file[mask]['s_probability'].values[0]
        except:
            continue
        if sst_prob < 0.3:
            print('No GOOD SKIP')
            continue
        else:
            print('COOL Go ahead')
        spt = np.zeros([1,1])
        spt[0,0] = float(spt_t/60.0)
        sst = np.zeros([1,1])
        sst[0,0] = float(sst_t/60.0)
        coda_end = np.zeros([1,1])
        coda_end[0,0] = float(sst_t/60.0)
        print(e_time)
        ref_pick_time = obspy.UTCDateTime(e_time[-27:]) + spt[0,0] * 60 - base_time
        print('REF TIME:{}'.format(ref_pick_time))
        seed_h5file = h5py_base_dir + '{}.hdf5'.format(sta[1][3:])
        with h5py.File(seed_h5file,'r') as f:
            dataset = f.get('data/'+e_time)
            data_t = np.array(dataset)
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            t_std = np.std(data_t, axis = 0, keepdims=True)
            t_std[t_std == 0] = 1.0
            data_t /= t_std
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            data_t_in = np.zeros([1,6000,3])
            data_t_in[0,:,:] = data_t
            
        for search_sta in search_list:
            # check if pick exists 
            #print(search_sta)
            search_ref_picks = s_eqt_dict[search_sta[1]]['S'] 
            if len(search_ref_picks) > 1:
                ref_key_id = bisect.bisect_left(search_ref_picks,ref_pick_time)
                try:
                    ref_left = np.abs(search_ref_picks[ref_key_id-1] - ref_pick_time)
                    ref_middle = np.abs(search_ref_picks[ref_key_id] - ref_pick_time)
                    ref_right = np.abs(search_ref_picks[ref_key_id+1] - ref_pick_time)
                    if ref_left < 60.0 or ref_middle < 60.0 or ref_right < 60.0:
                        print('Skipping {}'.format(search_sta[1]))
                        continue
                except:
                    pass
            else:
                pass
            h5py_base_dir_search = search_hdf5_prefix
            search_csvfile = h5py_base_dir_search + '{}.csv'.format(search_sta[1][3:])
            search_csvfile = pd.read_csv(search_csvfile)            
            keys = list(search_csvfile['trace_name'])
            if len(keys) < 2:
                print('Keys Error')
                continue
            prefix = keys[0][:-27]
            keys = [key[-27:] for key in keys]
            key_id = bisect.bisect_left(keys,e_time[-27:])
            
            try:
                search_keys = [keys[key_id-2],keys[key_id-1],keys[key_id],keys[key_id+1],keys[key_id+2]]
            except:
                print('Search_keys Error')
                continue
            # calculate key search range

            search_h5file =  h5py_base_dir_search + '{}.hdf5'.format(search_sta[1][3:])
            t_update_list = list()
            t_update_list_prob = list()
            with h5py.File(search_h5file,'r') as f:
                max_pred_amp = 0
                for search_key in search_keys:

                    t_search_key = prefix + search_key
                    dataset = f.get('data/'+t_search_key)
                    data_s = np.array(dataset)
                    data_s -= np.mean(data_s, axis=0 ,keepdims=True)
                    t_std = np.std(data_s, axis = 0, keepdims=True)
                    t_std[t_std == 0] = 1.0
                    data_s /= t_std

                    data_s_in = np.zeros([1,6000,3])
                    data_s_in[0,:,:] = data_s

                    encoded_t = encode_model.predict(data_t_in)
                    encoded_s = encode_model.predict(data_s_in)

                    siamese_input_list = list()

                    for rdx in range(len(RSRN_lengths)):
                        temp_length = float(RSRN_lengths[rdx])
                        template_s = int(spt[0,0]*temp_length) - 1
                        template_e = int(coda_end[0,0]*temp_length) + 1
                        template_w = int(template_e - template_s)
                        encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                        encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(RSRN_channels[rdx])])
                        encoded_s[rdx] = encoded_s[rdx].reshape([1,int(RSRN_lengths[rdx]),1,int(RSRN_channels[rdx])])

                        # channel normalization
                        for channel_dx in range(int(RSRN_channels[rdx])):
                            encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                            half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                            #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])

                            encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                            encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]

                            encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_s[rdx][0,:,0,channel_dx] /= t_max

                            encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                            encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_t[rdx][0,:,0,channel_dx] /= t_max

                        siamese_input_list.append(encoded_t[rdx])
                        siamese_input_list.append(encoded_s[rdx])

                    #print('RSRN Channel_normal OK')
                    if if_encoder_concate == 1:
                        for rdx in range(len(RSRN_lengths), len(RSRN_lengths) + len(encoder_encoded_list)):
                            rdx_2 = rdx - len(RSRN_lengths) 
                            temp_length = float(encoder_encoded_lengths[rdx_2])
                            template_s = int(spt[0,0]*temp_length) - 1
                            template_e = int(coda_end[0,0]*temp_length) + 1
                            template_w = int(template_e - template_s)
                            #print('Concate 1 OK')
                            encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                            encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(encoder_encoded_channels[rdx_2])])
                            encoded_s[rdx] = encoded_s[rdx].reshape([1,int(encoder_encoded_lengths[rdx_2]),1,int(encoder_encoded_channels[rdx_2])])
                            #print('Concate 2 OK')
                            # channel normalization
                            for channel_dx in range(int(encoder_encoded_channels[rdx_2])):
                                encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                                half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                                #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])
                                #print('Concate 3 OK')
                                encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                                encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]
                                #print('Concate 4 OK')
                                encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_s[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 5 OK')
                                encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                                encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_t[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 6 OK')
                            siamese_input_list.append(encoded_t[rdx])
                            siamese_input_list.append(encoded_s[rdx])
                    pred_res = siamese_model.predict(siamese_input_list)
                    siamese_s = np.argmax(pred_res[-1][0,:,0,0])
                    pred_amp = pred_res[-1][0,siamese_s,0,0]
                    
                    if pred_amp > 0.1 and siamese_s > 800 and siamese_s < 5200:
                        siamese_s_time = obspy.UTCDateTime(search_key) + siamese_s * 0.01
                        t_update_time  = siamese_s_time - base_time
                        t_diff_time = siamese_s_time - (obspy.UTCDateTime(e_time[-27:]) + sst[0,0] * 60)
                        #r_snr = np.max(np.abs(data_s_in[0,siamese_s-50:siamese_s,0]))/np.max(np.abs(data_s_in[0,siamese_s:siamese_s+100,0]))
                        
                        if np.abs(t_diff_time) < 12:
                            pass
                        else:
                            #print(t_diff_time)
                            continue

                        max_pred_amp = pred_amp
                        s_pred_amp = pred_amp
                        siamese_s_time = obspy.UTCDateTime(search_key) + siamese_s * 0.01
                        t_update_time  = siamese_s_time - base_time
  
                        t_update_list.append(t_update_time)
                        t_update_list_prob.append(pred_amp)
                if len(t_update_list) > 0:
                    max_arg = np.argmax(t_update_list_prob)
                    t_update_time = t_update_list[max_arg]
                    bisect.insort(s_eqt_dict[search_sta[1]]['S'], t_update_time)
                    print('{}'.format(t_update_time))
    """
    plt.figure(figsize=(6,6))
    plt.scatter(sta[3],sta[2],color='r')
    for t_sta in search_list:
        plt.scatter(t_sta[3],t_sta[2],color='b')
    plt.xlim([-118.8,-118.0])
    plt.ylim([34.0,34.8])
    plt.show()
    plt.close()
    """
    # open search list csv and hdf5 file
    # for all high P
    #     for all search stations
    #        for all time within range
    #           calcualte P
    #           keep highest Prob
    #           check insert Point
    #           

On 1 of 41
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
ALP_CI_HH_2010-01-04T20:08:00.002700Z
REF TIME:331702.9427
Skipping CI.VCS
Skipping CI.LEO
Skipping CI.STT
Skipping CI.BTP
COOL Go ahead
ALP_CI_HH_2010-01-04T20:11:30.002700Z
REF TIME:331909.5327
Skipping CI.VCS
Skipping CI.LEO
Skipping CI.STT
Skipping CI.BTP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
ALP_CI_HH_2010-01-13T20:31:30.002700Z
REF TIME:1110695.9927
Skipping CI.STT
No GOOD SKIP
COOL Go ahead
ALP_CI_HH_2010-01-14T12:10:30.002700Z
REF TIME:1167033.1527
Skipping CI.VCS
Skipping CI.LEO
Skipping CI.STT
Skipping CI.BTP
COOL Go ahead
ALP_CI_HH_2010-01-14T12:57:30.002700Z
REF TIME:1169858.9827
Skipping CI.VCS
Skipping CI.LEO
COOL Go ahead
ALP_CI_HH_2010-01-14T19:07:00.002700Z
REF TIME:1192027.9827
Skipping CI.STT
COOL Go ahead
ALP_CI_HH_2010-01-14T19:07:00.002700Z
REF TIME:1192027.9827
Skipping CI.STT
No GOOD SKIP
COOL Go ahead
ALP_CI_HH_2010-01-14T21:34:30.002700Z
REF TI

Skipping CI.IR2
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
LEO_CI_EH_2010-01-03T19:51:30.009900Z
REF TIME:244299.9799
COOL Go ahead
LEO_CI_EH_2010-01-03T20:40:00.009900Z
REF TIME:247204.0399
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
LEO_CI_EH_2010-01-04T20:52:30.005800Z
REF TIME:334385.8258
Skipping CI.IR2
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
LEO_CI_EH_2010-01-07T09:12:00.000000Z
REF TIME:551530.6
Skipping CI.STT
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
LEO_CI_EH_2010-01-08T14:50:00.000000Z
REF TIME:658208.06
Skipping CI.STT
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
LEO_CI_EH_2010-01-10T13:48:00.000000Z
REF TIME:827335.06
Skipping CI

Skipping CI.SMS
Skipping CI.DJJ
Skipping CI.LFP
Skipping CI.MWC
Skipping CI.DJJB
Skipping CI.PASC
COOL Go ahead
DEC_CI_HH_2010-01-02T03:15:30.008500Z
REF TIME:98160.4885
Skipping CI.GSA
Skipping CI.WNS
Skipping CI.MIKB
Skipping CI.USC
Skipping CI.NOT
Skipping CI.SMS
Skipping CI.CBC
Skipping CI.QUG
Skipping CI.DJJ
Skipping CI.LFP
Skipping CI.LCG
Skipping CI.MWC
Skipping CI.DJJB
Skipping CI.PASC
Skipping CI.GVR
Skipping CI.MIK
Skipping CI.HLL
COOL Go ahead
DEC_CI_HH_2010-01-04T20:08:00.008500Z
REF TIME:331710.5185
Skipping CI.GSA
Skipping CI.WNS
Skipping CI.MIKB
Skipping CI.SMS
Skipping CI.SMF
Skipping CI.CBC
Skipping CI.QUG
Skipping CI.DJJ
Skipping CI.LFP
Skipping CI.LCG
Skipping CI.MWC
Skipping CI.DJJB
Skipping CI.IR2
Skipping CI.PASC
Skipping CI.OAT
Skipping CI.GVR
Skipping CI.HLL
331728.86
COOL Go ahead
DEC_CI_HH_2010-01-04T20:08:30.008500Z
REF TIME:331714.0285
Skipping CI.GSA
Skipping CI.WNS
Skipping CI.MIKB
Skipping CI.SMS
Skipping CI.SMF
Skipping CI.CBC
Skipping CI.QUG
Skipping CI

Skipping CI.SMV
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
NOT_CI_EH_2010-01-10T00:00:00.000000Z
REF TIME:777608.13
Skipping CI.SMV
Skipping CI.PDE
Skipping CI.OAT
Skipping CI.RINB
Skipping CI.SPF
COOL Go ahead
NOT_CI_EH_2010-01-10T13:39:30.000000Z
REF TIME:826785.42
Skipping CI.SMV
826789.2613
Skipping CI.OAT
COOL Go ahead
NOT_CI_EH_2010-01-10T15:52:30.000000Z
REF TIME:834760.34
834774.0888
Search_keys Error
Skipping CI.PDE
No GOOD SKIP
COOL Go ahead
NOT_CI_EH_2010-01-11T16:20:30.000000Z
REF TIME:922839.88
Skipping CI.DEC
Search_keys Error
Skipping CI.DJJB
COOL Go ahead
NOT_CI_EH_2010-01-11T16:20:30.000000Z
REF TIME:922839.88
Skipping CI.DEC
Search_keys Error
Skipping CI.DJJB
COOL Go ahead
NOT_CI_EH_2010-01-11T16:48:30.000000Z
REF TIME:924526.9
Skipping CI.WSS
Search_keys Error
COOL Go ahead
NOT_CI_EH_2010-01-11T19:31:00.000000Z
REF TIME:934265.93
Skipping CI.DEC
Search_keys Error
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
NOT

Skipping CI.DEC
Skipping CI.NOT
Skipping CI.LCG
Skipping CI.DJJB
Skipping CI.PASC
COOL Go ahead
DJJ_CI_HH_2010-01-01T04:33:30.008800Z
REF TIME:16465.2988
Skipping CI.WNS
Skipping CI.DEC
Skipping CI.SMS
Skipping CI.LCG
Skipping CI.AGO
Skipping CI.DJJB
Skipping CI.PASC
Skipping CI.SPF
COOL Go ahead
DJJ_CI_HH_2010-01-01T06:17:30.008800Z
REF TIME:22660.3788
Skipping CI.WSS
Skipping CI.LCG
Skipping CI.DJJB
Skipping CI.RINB
COOL Go ahead
DJJ_CI_HH_2010-01-01T10:47:30.008800Z
REF TIME:38885.3388
Skipping CI.NOT
Skipping CI.SMS
Skipping CI.DJJB
Skipping CI.SPF
COOL Go ahead
DJJ_CI_HH_2010-01-01T22:20:00.008800Z
REF TIME:80428.8688
Skipping CI.DEC
Skipping CI.SMS
Skipping CI.LFP
Skipping CI.DJJB
Skipping CI.PASC
Skipping CI.SPF
COOL Go ahead
DJJ_CI_HH_2010-01-01T22:20:30.008800Z
REF TIME:80431.6188
Skipping CI.DEC
Skipping CI.SMS
Skipping CI.LFP
Skipping CI.DJJB
Skipping CI.PASC
Skipping CI.SPF
COOL Go ahead
DJJ_CI_HH_2010-01-02T03:15:30.008800Z
REF TIME:98156.6788
Skipping CI.WNS
98165.0965
Sk

...

Skipping CI.PASC
Skipping CI.SPF
COOL Go ahead
DJJB_CI_HN_2010-01-05T04:49:30.004400Z
REF TIME:363013.2344
Skipping CI.WNS
Skipping CI.WSS
Skipping CI.DEC
Skipping CI.NOT
Skipping CI.SMS
Skipping CI.BLC
Skipping CI.DJJ
Skipping CI.LFP
Skipping CI.LCG
Skipping CI.AGO
Skipping CI.PASC
Skipping CI.SPF
Skipping CI.HLL
COOL Go ahead
DJJB_CI_HN_2010-01-05T05:14:00.004400Z
REF TIME:364476.5444
Skipping CI.USC
Skipping CI.DEC
Skipping CI.BLC
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.AGO
Skipping CI.PASC
Skipping CI.SPF
COOL Go ahead
DJJB_CI_HN_2010-01-05T07:59:00.004400Z
REF TIME:374383.5544
Skipping CI.WNS
Skipping CI.RIN
Skipping CI.BLC
Skipping CI.DJJ
Skipping CI.LFP
Skipping CI.AGO
Skipping CI.RINB
Skipping CI.SPF
COOL Go ahead
DJJB_CI_HN_2010-01-05T07:59:00.004400Z
REF TIME:374383.5544
Skipping CI.WNS
Skipping CI.RIN
Skipping CI.BLC
Skipping CI.DJJ
Skipping CI.LFP
Skipping CI.AGO
Skipping CI.RINB
Skipping CI.SPF
No GOOD SKIP
COOL Go ahead
DJJB_CI_HN_2010-01-05T14:44:30.004400Z
REF TIME:

Skipping CI.BLC
Skipping CI.PDE
No GOOD SKIP
COOL Go ahead
IR2_CI_EH_2010-01-06T23:24:00.000200Z
REF TIME:516246.9202
Skipping CI.LEO
Skipping CI.VVD
Skipping CI.BLC
Skipping CI.PDE
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
IR2_CI_EH_2010-01-07T06:03:00.000000Z
REF TIME:540209.28
Skipping CI.BLC
No GOOD SKIP
COOL Go ahead
IR2_CI_EH_2010-01-07T08:00:30.000000Z
REF TIME:547259.51
Skipping CI.LEO
Skipping CI.VVD
Skipping CI.DEC
Skipping CI.NOT
Skipping CI.BLC
Skipping CI.LFP
547262.39
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
IR2_CI_EH_2010-01-07T18:23:30.000000Z
REF TIME:584620.33
Skipping CI.NOT
Skipping CI.BLC
Skipping CI.PDE
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
IR2_CI_EH_2010-01-08T05:21:30.000000Z
REF TIME:624114.02
Skipping CI.VCS
Skipping CI.LEO
Skipping CI.DEC
Skipping CI.NOT
Skipping CI.BLC
Skipping CI.PDE
Skipping CI.OAT
COOL Go ahead
IR2_CI_EH_2010-01-08T09:06:30.000000Z
REF TIME:637641.48
Skipping CI.VVD
Skipping CI.OAT
No GOOD SKIP
COOL Go ahead
IR2_C

Skipping CI.WNS
Skipping CI.MIKB
Skipping CI.RHC
Skipping CI.DEC
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.MWC
Skipping CI.DJJB
Skipping CI.GVR
Skipping CI.MIK
Skipping CI.HLL
COOL Go ahead
PASC_CI_HH_2010-01-05T06:32:30.008300Z
REF TIME:369168.0683
Skipping CI.RHC
Skipping CI.DEC
Skipping CI.LCG
Skipping CI.MWC
Skipping CI.RUS
Skipping CI.GVR
COOL Go ahead
PASC_CI_HH_2010-01-05T08:16:30.008300Z
REF TIME:375393.9883
Skipping CI.MIK
Skipping CI.CAC
COOL Go ahead
PASC_CI_HH_2010-01-05T15:17:30.008300Z
REF TIME:400695.5883
400704.7227
Skipping CI.RHC
Skipping CI.DEC
Skipping CI.MWC
Skipping CI.RUS
Skipping CI.GVR
COOL Go ahead
PASC_CI_HH_2010-01-07T08:00:30.008300Z
REF TIME:547261.0983
Skipping CI.WNS
Skipping CI.USC
Skipping CI.DEC
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.MWC
Skipping CI.DJJB
COOL Go ahead
PASC_CI_HH_2010-01-08T05:21:30.008300Z
REF TIME:624110.1683
Skipping CI.RHC
Skipping CI.DEC
Skipping CI.SMF
624128.5588
Skipping CI.LCG
Skipping CI.MWC
Skipping CI.DJJB
624111.83


Skipping CI.AGO
Skipping CI.DJJB
COOL Go ahead
SPF_CI_HH_2010-01-05T07:59:00.000400Z
REF TIME:374382.8504
Skipping CI.WNS
Skipping CI.RIN
Skipping CI.SMV
Skipping CI.BLC
Skipping CI.DJJ
Skipping CI.AGO
Skipping CI.DJJB
Skipping CI.RINB
COOL Go ahead
SPF_CI_HH_2010-01-06T00:00:00.000400Z
REF TIME:432006.1604
Skipping CI.WNS
Skipping CI.RIN
Skipping CI.LCG
Skipping CI.DJJB
Skipping CI.RINB
COOL Go ahead
SPF_CI_HH_2010-01-06T00:12:30.000400Z
REF TIME:432752.0904
Skipping CI.WNS
No GOOD SKIP
COOL Go ahead
SPF_CI_HH_2010-01-06T21:09:00.000400Z
REF TIME:508147.9804
Skipping CI.SMS
Skipping CI.AGO
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
SPF_CI_HH_2010-01-07T08:00:30.000400Z
REF TIME:547260.7004
Skipping CI.WNS
Skipping CI.NOT
Skipping CI.SMS
Skipping CI.BLC
Skipping CI.DJJ
Skipping CI.LCG
Skipping CI.DJJB
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
SPF_CI_HH_2010-01-07T22:05:00.000400Z
REF TIME:597928.3204
Skipping CI.WNS
Skipping CI.RIN
Skipping CI.SMV
Skipping CI.LCG
S

Skipping CI.QAL
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
STT_CI_EH_2010-01-13T00:23:30.000200Z
REF TIME:1038260.1302
Skipping CI.QAL
COOL Go ahead
STT_CI_EH_2010-01-13T02:57:30.000200Z
REF TIME:1047481.7202
Skipping CI.QAL
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
STT_CI_EH_2010-01-13T23:01:00.000200Z
REF TIME:1119683.3202
Skipping CI.QAL
Skipping CI.BTP
No GOOD SKIP
No GOOD SKIP
COOL Go ahead
STT_CI_EH_2010-01-13T23:58:00.000200Z
REF TIME:1123086.8902
Skipping CI.ALP
Skipping CI.LEO
Skipping CI.QAL
COOL Go ahead
STT_CI_EH_2010-01-13T23:58:00

In [30]:
np.save('seqt_KLTG_1116_S.npy',s_eqt_dict)

In [47]:
for sta_key in s_eqt_dict.keys():
    cur_file = '/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/SEQTRes/20100101/{}.S.txt'.format(sta_key)
    f = open(cur_file,'w')
    arrival_dx = 0
    pre_arrival = -1
    for arrival in s_eqt_dict[sta_key]['S']:
        if arrival_dx == 0:
            pre_arrival = arrival
            arrival_dx += 1
        else:
            if np.abs(pre_arrival - arrival) < 1:
                continue
            else:
                pre_arrival = arrival
                
        S_res = '{:.3f} 0.80 0.00000000'.format(arrival)
        f.write(S_res+'\n')
    f.close()

In [25]:
for sta_key in s_eqt_dict.keys():
    cur_file = '/home/Public/SiameseEQTransformer/notebooks/XJ_Events/REAL_SEQT_619-621/{}.S.txt'.format(sta_key)
    f = open(cur_file,'w')
    for arrival in s_eqt_dict[sta_key]['S']:
        S_res = '{:.3f} 0.90 0.00000000'.format(arrival)
        f.write(S_res+'\n')
    f.close()

In [ ]:
# test 10
plot_dx = 0
sta_search_dx = 0
plot_num = 0
for sta in station_list:
    sta_search_dx += 1
    print('On {} of {}'.format(sta_search_dx, len(station_list)))
    # select stations to search
    search_list = get_search_station_list(sta, station_list, max_search_distance)
    seed_csv = '/run/media/root/Backup Plus/XJ2SEQT/{}_detections/{}_outputs/X_prediction_results.csv'.format(time_label,sta[1][3:])
    seed_csv_file = pd.read_csv(seed_csv)
    for e_time in seed_csv_file['file_name']:
        mask = (seed_csv_file['file_name'] == e_time)
        try:
            spt_t = obspy.UTCDateTime(seed_csv_file[mask]['p_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            sst_t = obspy.UTCDateTime(seed_csv_file[mask]['s_arrival_time'].values[0]) - obspy.UTCDateTime(e_time[-27:])
            spt_prob = seed_csv_file[mask]['p_probability'].values[0]
            sst_prob = seed_csv_file[mask]['s_probability'].values[0]
        except:
            continue
        if spt_prob < 0.6:
            continue
        spt = np.zeros([1,1])
        spt[0,0] = float(spt_t/60.0)
        sst = np.zeros([1,1])
        sst[0,0] = float(sst_t/60.0)
        coda_end = np.zeros([1,1])
        coda_end[0,0] = float(sst_t/60.0)
        print(e_time)
        ref_pick_time = obspy.UTCDateTime(e_time[-27:]) + sst[0,0] * 60 - obspy.UTCDateTime(2018,6,19)
        
        
        
        seed_h5file = h5py_base_dir + '{}.hdf5'.format(sta[1][3:])
        with h5py.File(seed_h5file,'r') as f:
            dataset = f.get('data/'+e_time)
            data_t = np.array(dataset)
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            t_std = np.std(data_t, axis = 0, keepdims=True)
            t_std[t_std == 0] = 1.0
            data_t /= t_std
            data_t -= np.mean(data_t, axis=0 ,keepdims=True)
            data_t_in = np.zeros([1,6000,3])
            data_t_in[0,:,:] = data_t
            
        for search_sta in search_list:
            # check if pick exists 
            #print(search_sta)
            search_ref_picks = s_eqt_dict[search_sta[1]]['S'] 
            if len(search_ref_picks) > 1:
                ref_key_id = bisect.bisect_left(search_ref_picks,ref_pick_time)
                try:
                    ref_left = np.abs(search_ref_picks[ref_key_id-1] - ref_pick_time)
                    ref_middle = np.abs(search_ref_picks[ref_key_id] - ref_pick_time)
                    ref_right = np.abs(search_ref_picks[ref_key_id+1] - ref_pick_time)
                    if ref_left < 6.0 or ref_middle < 6.0 or ref_right < 6.0:
                        print('Skipping {}'.format(search_sta[1]))
                        continue
                except:
                    pass
            else:
                pass
            
            search_csvfile = h5py_base_dir + '{}.csv'.format(search_sta[1][3:])
            search_csvfile = pd.read_csv(search_csvfile)
            keys = list(search_csvfile['trace_name'])
            prefix = keys[0][:-27]
            keys = [key[-27:] for key in keys]
            key_id = bisect.bisect_left(keys,e_time[-27:])
            
            try:
                search_keys = [keys[key_id-2],keys[key_id-1],keys[key_id],keys[key_id+1],keys[key_id+2]]
            except:
                continue
            # calculate key search range

            search_h5file =  h5py_base_dir + '{}.hdf5'.format(search_sta[1][3:])
            t_update_list = list()
            t_update_list_prob = list()
            with h5py.File(search_h5file,'r') as f:
                max_pred_amp = 0
                for search_key in search_keys:

                    t_search_key = prefix + search_key
                    dataset = f.get('data/'+t_search_key)
                    data_s = np.array(dataset)
                    data_s -= np.mean(data_s, axis=0 ,keepdims=True)
                    t_std = np.std(data_s, axis = 0, keepdims=True)
                    t_std[t_std == 0] = 1.0
                    data_s /= t_std

                    data_s_in = np.zeros([1,6000,3])
                    data_s_in[0,:,:] = data_s

                    encoded_t = encode_model.predict(data_t_in)
                    encoded_s = encode_model.predict(data_s_in)

                    siamese_input_list = list()

                    for rdx in range(len(RSRN_lengths)):
                        temp_length = float(RSRN_lengths[rdx])
                        template_s = int(spt[0,0]*temp_length) - 1
                        template_e = int(coda_end[0,0]*temp_length) + 1
                        template_w = int(template_e - template_s)
                        encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                        encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(RSRN_channels[rdx])])
                        encoded_s[rdx] = encoded_s[rdx].reshape([1,int(RSRN_lengths[rdx]),1,int(RSRN_channels[rdx])])

                        # channel normalization
                        for channel_dx in range(int(RSRN_channels[rdx])):
                            encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                            half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                            #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])

                            encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                            encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]

                            encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_s[rdx][0,:,0,channel_dx] /= t_max

                            encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                            encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                            encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                            t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                            if t_max < 0.01:
                                t_max = 1
                            encoded_t[rdx][0,:,0,channel_dx] /= t_max

                        siamese_input_list.append(encoded_t[rdx])
                        siamese_input_list.append(encoded_s[rdx])

                    #print('RSRN Channel_normal OK')
                    if if_encoder_concate == 1:
                        for rdx in range(len(RSRN_lengths), len(RSRN_lengths) + len(encoder_encoded_list)):
                            rdx_2 = rdx - len(RSRN_lengths) 
                            temp_length = float(encoder_encoded_lengths[rdx_2])
                            template_s = int(spt[0,0]*temp_length) - 1
                            template_e = int(coda_end[0,0]*temp_length) + 1
                            template_w = int(template_e - template_s)
                            #print('Concate 1 OK')
                            encoded_t[rdx] = encoded_t[rdx][:,template_s:template_e,:]/float(template_w)
                            encoded_t[rdx] = encoded_t[rdx].reshape([1,template_w,1,int(encoder_encoded_channels[rdx_2])])
                            encoded_s[rdx] = encoded_s[rdx].reshape([1,int(encoder_encoded_lengths[rdx_2]),1,int(encoder_encoded_channels[rdx_2])])
                            #print('Concate 2 OK')
                            # channel normalization
                            for channel_dx in range(int(encoder_encoded_channels[rdx_2])):
                                encoded_s[rdx][0,:,0,channel_dx] -= np.max(encoded_s[rdx][0,:,0,channel_dx])
                                half_window_len = int( 200.0*temp_length/6000.0   ) + 1
                                #window_mean = np.mean(encoded_s[rdx][0,half_window_len:-half_window_len,0,channel_dx])
                                #print('Concate 3 OK')
                                encoded_s[rdx][0,:half_window_len,0,channel_dx] = encoded_s[rdx][0,half_window_len,0,channel_dx]
                                encoded_s[rdx][0,-half_window_len:,0,channel_dx] =  encoded_s[rdx][0,-half_window_len,0,channel_dx]
                                #print('Concate 4 OK')
                                encoded_s[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_s[rdx][0,:,0,channel_dx] -= np.mean(encoded_s[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_s[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_s[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 5 OK')
                                encoded_t[rdx][0,:,0,channel_dx] -= np.max(encoded_t[rdx][0,:,0,channel_dx])
                                encoded_t[rdx][0,:,0,channel_dx] *= -1.0
                                encoded_t[rdx][0,:,0,channel_dx] -= np.mean(encoded_t[rdx][0,:,0,channel_dx])
                                t_max = np.max(np.abs(encoded_t[rdx][0,:,0,channel_dx]))
                                if t_max < 0.01:
                                    t_max = 1
                                encoded_t[rdx][0,:,0,channel_dx] /= t_max
                                #print('Concate 6 OK')
                            siamese_input_list.append(encoded_t[rdx])
                            siamese_input_list.append(encoded_s[rdx])
                    pred_res = siamese_model.predict(siamese_input_list)
                    siamese_s = np.argmax(pred_res[-1][0,:,0,0])
                    pred_amp = pred_res[-1][0,siamese_s,0,0]
                    
                    if pred_amp > 0.3 and siamese_s > 200 and siamese_s < 5800:
                        siamese_s_time = obspy.UTCDateTime(search_key) + siamese_s * 0.01
                        t_update_time  = siamese_s_time - base_time
                        t_diff_time = siamese_s_time - (obspy.UTCDateTime(e_time[-27:]) + sst[0,0] * 60)
                        #r_snr = np.max(np.abs(data_s_in[0,siamese_s-50:siamese_s,0]))/np.max(np.abs(data_s_in[0,siamese_s:siamese_s+100,0]))
                        
                        if np.abs(t_diff_time) < 8:
                            pass
                        else:
                            print(t_diff_time)
                            continue

                        max_pred_amp = pred_amp
                        s_pred_amp = pred_amp
                        siamese_s_time = obspy.UTCDateTime(search_key) + siamese_s * 0.01
                        t_update_time  = siamese_s_time - base_time
                        plot_data_s = data_s_in
                        
                        #print(max_pred_amp)
                        #print(siamese_s_time)
                        #print(obspy.UTCDateTime(e_time[-27:]) + sst[0,0] * 60)


                        eqt_res = EqT_model.predict(plot_data_s)
                        eqt_s = np.argmax(eqt_res[2][0,:,0])
                        pred_amp = eqt_res[2][0,eqt_s,0]
                        if pred_amp > 0.1:
                            pass
                        else:
                            eqt_s = 0.0

                        plt.figure(figsize=(20,16))
                        pdx = 0
                        plt.plot(data_t_in[0,:,0]+pdx*20,color='k')
                        plt.plot(data_t_in[0,:,1]+pdx*20+4,color='k')
                        plt.plot(data_t_in[0,:,2]+pdx*20+8,color='k')
                        plt.plot([sst_t*100,sst_t*100],[pdx*20-10,pdx*20+10],color='g',linewidth=3)

                        pdx = 1
                        plt.plot(plot_data_s[0,:,0]+pdx*20,color='k')
                        plt.plot(plot_data_s[0,:,1]+pdx*20+4,color='k')
                        plt.plot(plot_data_s[0,:,2]+pdx*20+8,color='k')

                        plt.plot([eqt_s,eqt_s],[pdx*20-10,pdx*20+10],color='g',linewidth=3)

                        plt.plot([siamese_s,siamese_s],[pdx*20-7,pdx*20+7],color='r',linewidth=2)
                        plt.title('{}'.format(s_pred_amp),fontsize=30)
                        plt.savefig('t_figs/S_CHECK_{}_{}_{}.png'.format(sta[1][3:],search_sta[1][3:],plot_dx))
                        plot_dx += 1
                        plot_num += 1
                        plt.close()
            
                        t_update_list.append(t_update_time)
                        t_update_list_prob.append(pred_amp)
                if len(t_update_list) > 0:
                    max_arg = np.argmax(t_update_list_prob)
                    t_update_time = t_update_list[max_arg]
                    bisect.insort(s_eqt_dict[search_sta[1]]['S'], t_update_time)
                    print('{}'.format(t_update_time))
                    
            if plot_num > 10:
                break
        if plot_num > 10:
            break
    if plot_num > 10:
        break
                           
    """
    plt.figure(figsize=(6,6))
    plt.scatter(sta[3],sta[2],color='r')
    for t_sta in search_list:
        plt.scatter(t_sta[3],t_sta[2],color='b')
    plt.xlim([-118.8,-118.0])
    plt.ylim([34.0,34.8])
    plt.show()
    plt.close()
    """
    # open search list csv and hdf5 file
    # for all high P
    #     for all search stations
    #        for all time within range
    #           calcualte P
    #           keep highest Prob
    #           check insert Point
    #           

In [24]:
len(s_eqt_dict.keys())

41

In [27]:
for sta_key in s_eqt_dict.keys():
    s_times = s_eqt_dict[sta_key]['S']
    savefile = open('/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/SEQTRes/20100101/{}.S.txt'.format(sta_key),'w')
    for s_t in s_times:
        savefile.write('{:.3f} 0.80000 0.00000000\n'.format(s_t))
    savefile.close()

[10554.053, 98182.773, 331719.003, 331924.823, 363024.3927, 586317.2127, 683953.173, 827337.2827, 923929.493, 935598.623, 948284.173, 948875.863, 959811.763, 1003383.563, 1040201.7527, 1110698.903, 1120453.533, 1152948.5527, 1167049.823, 1169877.093, 1192038.433, 1192066.193, 1200155.233, 1200905.763, 1201704.613, 1201790.903]
[10557.441, 12380.8707, 98180.121, 269739.9107, 274434.8707, 331722.491, 331928.301, 340659.961, 362733.861, 363024.821, 381381.9807, 428306.491, 448451.5907, 477696.5507, 737807.621, 779382.731, 788168.771, 796220.631, 810587.5007, 827338.7207, 829394.4707, 859264.431, 900971.1207, 948276.571, 948868.441, 959804.691, 978229.4307, 984971.041, 1040191.691, 1152945.201, 1167055.921, 1169882.931, 1178955.541]
[98173.5107, 331733.6607, 411220.721, 526445.2707, 948865.8307, 959801.741, 959801.7507]
[16466.8848, 98168.217, 190098.0767, 267406.4807, 331735.7507, 362943.6827, 363018.753, 369167.9227, 439134.423, 521057.099, 521089.039, 522058.929, 526442.039, 526469.239,

In [ ]:
keep_t = None
for sta_key in s_eqt_dict.keys():
    S_EqT_S_list = list()
    print(sta_key)
    if len(s_eqt_dict[sta_key]['S']) > 0:
        pass
    else:
        continue
    s_times = s_eqt_dict[sta_key]['S']
    s_probs = s_eqt_dict[sta_key]['S_Prob']
    prev_file = '/mnt/BYEB/LA_JAN/LA_DATA/PostProcess/SEQTRes/20100101/{}.S.txt'.format(sta_key)
    if os.path.exists(prev_file):
        f = open(prev_file,'r')
        for line in f.readlines():
            if len(line) > 3:
                s_times.append(float(line.split(' ')[0]))
                s_probs.append(0.1)
        f.close()
    
    indexs = np.argsort(s_times)
    for idx in range(len(indexs)-1):
        t_id_0 =  indexs[idx]
        t_id_1 = indexs[idx+1]
        if s_times[t_id_1] - s_times[t_id_0] < 3.0:
            if s_probs[t_id_1] >= s_probs[t_id_0]:
                keep_t = s_times[t_id_1]
                keep_prob = s_probs[t_id_1]
            else:
                keep_t = s_times[t_id_0]
                keep_prob = s_probs[t_id_0]
        elif keep_t is None:
            S_EqT_S_list.append('{:.3f} {:.5f} 0.00000000'.format(s_times[t_id_0], s_probs[t_id_0]))
        elif idx == len(indexs)-2:
            S_EqT_S_list.append('{:.3f} {:.5f}  0.00000000'.format(s_times[t_id_0], s_probs[t_id_0]))
            S_EqT_S_list.append('{:.3f} {:.5f}  0.00000000'.format(s_times[t_id_1], s_probs[t_id_1]))
        else:
            S_EqT_S_list.append('{:.3f} {:.5f}  0.00000000'.format(keep_t, keep_prob))
            keep_t = None
    cur_file = '/root/Desktop/Public/SiameseEQTransformer/notebooks/EQT2REAL_PICK_SEQT/{}.P.txt'.format(sta_key)
    f = open(cur_file,'w')
    for S_res in S_EqT_S_list:
        f.write(S_res+'\n')
    f.close()
    #print(S_EqT_S_list)
    #break

In [ ]:
print(S_EqT_S_list)